In [1]:
import random
import math
import matplotlib.pyplot as plt
import time
import numpy as np
from point import Point
import json

# Load the Layout Pool from Static Generation

In [2]:
with open("data/static_layout_pool.json", 'r') as openfile:
    json_object = json.load(openfile)

layouts = [[Point(p[0], p[1]) for p in lay] for lay in json_object]

# Dynamic Trial Shuffling
- We shuffle the N trials such that the boundary visibility alternates each time.
- We read N/2 trials from static generation.
- Boundary visibility is alternating, so we simply assume border_visibility = i % 2

## Generation

- At least K = 5 trials between trials of the same pairing.
- We make sure that between adjacent pairs, the ending location and starting location is always opposite.
    - Also, make sure these distances obey MIN_DIST_PATH

In [15]:
NUM_SHUFFLES = 50
DIST_BETWEEN_COPY = 5 # K
LAYOUT_COUNT = len(layouts)

In [18]:
def generate_random_shuffling():
    def random_half_shuffle():
        l = list(range(len(layouts)))
        random.shuffle(l)
        return l
    
    return [random_half_shuffle(), random_half_shuffle()]

def is_valid_shuffle(order):
    # At least K = 5 trials between trials of the same pairing.
    # since we generate this as half and half, we only need to check the first and last K elements
    for i in range(DIST_BETWEEN_COPY):
        if order[0][-1] in order[1][:DIST_BETWEEN_COPY-i]:
            return False
    
    # each pair gets one occurence with border, one occurence without border
    with_border = set()
    for i, l in enumerate(order[0] + order[1]):
        if i % 2 == 1:
            if l in with_border:
                return False
            with_border.add(l)
            
        
        
def generate_valid_shuffle():
    shuffle = generate_random_shuffling()
    while not is_valid_shuffle(shuffle):
        shuffle = generate_random_shuffling()
    return shuffle

In [19]:
shuffles = [generate_valid_shuffle() for i in range(NUM_SHUFFLES)]
shuffles

[[44,
  16,
  29,
  20,
  25,
  12,
  28,
  40,
  7,
  30,
  23,
  48,
  0,
  37,
  24,
  35,
  31,
  39,
  42,
  45,
  13,
  8,
  15,
  6,
  17,
  49,
  18,
  38,
  27,
  14,
  43,
  21,
  33,
  10,
  32,
  5,
  2,
  4,
  36,
  1,
  19,
  11,
  34,
  26,
  3,
  47,
  46,
  9,
  22,
  41,
  42,
  23,
  24,
  35,
  49,
  39,
  0,
  40,
  13,
  28,
  31,
  25,
  17,
  18,
  48,
  14,
  15,
  19,
  21,
  3,
  10,
  45,
  29,
  12,
  27,
  41,
  38,
  33,
  36,
  8,
  7,
  34,
  20,
  1,
  11,
  9,
  32,
  30,
  2,
  26,
  47,
  37,
  5,
  6,
  4,
  44,
  43,
  46,
  16,
  22],
 [34,
  33,
  25,
  40,
  1,
  28,
  46,
  45,
  30,
  36,
  47,
  7,
  32,
  13,
  18,
  6,
  21,
  9,
  44,
  38,
  22,
  43,
  42,
  41,
  17,
  4,
  3,
  24,
  48,
  19,
  5,
  23,
  11,
  20,
  10,
  0,
  31,
  16,
  27,
  8,
  29,
  2,
  14,
  12,
  37,
  26,
  35,
  15,
  49,
  39,
  45,
  11,
  7,
  37,
  40,
  18,
  42,
  22,
  32,
  5,
  35,
  44,
  4,
  1,
  16,
  24,
  14,
  17,
  27,
  3,
  8,
  26,
  1

## Checks

- We make sure that apmprox. half the pairs happen in A->B, and the other half happens in B->A
- We want to check the number of objects within the same side across a sliding window of 5 trials.

# Objects
- 5 different shapes (circle, cube, cylinder, pyramid, star)
- 5 different colors from a color-blind-friendly palette.
-> 25 different combinations.

Any object generated must not have been in the last 10 shown object.